# Authentication

Authentication works via modifiers to the request (it is possible to add or change HTTP request headers).
This is set at the client level and applies to all APIs and supported paths.

## Note:

This notebook does not contact an actual OGC API endpoint, it is only to verify that the requests are correctly modified.

For this purpose, it is necessary to be able to receive the request via a simple testing server, such as [simple-http-logger](https://github.com/CycodeLabs/simple-http-logger), which can be started like this

```bash
server.py 8081
```

## Define the modifier method

The method prints a line to show that it was called and adds a Bearer header.

In [ ]:
def set_token(**kwargs):
    print("Setting header")
    headers = kwargs.get('headers')
    if headers is not None:
        headers['Authorization'] = "Bearer 1234"


## Get the processes list as an example

In [ ]:
from ogc_api_client.api_client import Configuration
from ogc_api_client.api_client_wrapper import ApiClientWrapper

namespace = "acme"

# Testing host (localhost), not an actual OGC API endpoint:
ogc_api_endpoint = f"http://localhost:8081/{namespace}/ogc-api"

configuration = Configuration(
    host=ogc_api_endpoint,
)
client = ApiClientWrapper(configuration=configuration, modifier=set_token)

try:
    processes = client.get_processes()
except Exception as e:
    print("EXCEPTION (expected): {0}".format(str(e)))


## Explanation

The `modifier` parameter for the `ApiClientWrapper` constructor is a method that needs to accept a `headers` dictionary as a keyword argument.

That method can add or modify the headers, which are then sent to the actual OGC API (in this notebook only to a testing server). Since the response is empty and not a process list, the API call will fail, but we are only interested in the correct request.

The server should receive an Authorization header:

```
Host: localhost:8081
Accept-Encoding: identity
Authorization: Bearer 1234
Accept: application/json
User-Agent: OpenAPI-Generator/1.0.0/python

127.0.0.1 - - [03/Jul/2025 17:00:37] "GET /acme/ogc-api/processes HTTP/1.1" 200 -
```

This flexible approach can be used for all sorts of header-based authentication, in particular API key or Bearer authentication.